In [1]:
from pcd_dataset.kitti_dataset import KittiDataset

dataset_path = 'dataset_baby/'
sequence = '00'
image_instances_path = 'pipeline_baby/vfm-labels/sam/00/'
kitti = KittiDataset(dataset_path, sequence, image_instances_path)

In [2]:
start_index = 20
end_index = 23
cam_name = 'cam2'

In [3]:
from services.label_service import get_map_not_zero_in_sphere

map, points2instances = get_map_not_zero_in_sphere(kitti, cam_name, start_index, end_index, 8, True)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [4]:
import copy

map_copy = copy.deepcopy(map)

min_bound = map_copy.get_min_bound()
max_bound = map_copy.get_max_bound()
downpcd_trace = map_copy.voxel_down_sample_and_trace(0.03, min_bound, max_bound, True)

In [5]:
list_int_vectors = downpcd_trace[2]
list_int_vectors[8:15]

[IntVector[10044, 10045],
 IntVector[10042, 10043],
 IntVector[10040, 10041],
 IntVector[10038, 10039],
 IntVector[10034, 10035],
 IntVector[10031],
 IntVector[10028]]

In [6]:
import numpy as np

# находим максимальное количество точек, объединенных в одну
# строим list_int_vectors_arrays --- список int_vectors, преобразованных в массив: IntVector[10053, 10054] -> [[10053, 10054]]
def find_max_col_and_list_int_vectors_arrays(list_int_vectors):
    maxc = 0
    list_int_vectors_arrays = []

    for i in range(len(list_int_vectors)):
        int_vector = list_int_vectors[i]
        int_vector_array = np.asarray(int_vector)
        if len(int_vector_array) > maxc:
            maxc = len(int_vector_array)
        list_int_vectors_arrays.append(int_vector_array)

    return maxc, list_int_vectors_arrays

maxc, list_int_vectors_arrays = find_max_col_and_list_int_vectors_arrays(list_int_vectors)

In [7]:
list_int_vectors_arrays[8:15]

[array([10044, 10045], dtype=int32),
 array([10042, 10043], dtype=int32),
 array([10040, 10041], dtype=int32),
 array([10038, 10039], dtype=int32),
 array([10034, 10035], dtype=int32),
 array([10031], dtype=int32),
 array([10028], dtype=int32)]

In [8]:
# строим 2d массив: в i--ой строке перечислены СТАРЫЕ точки, которые были объединены для создания НОВОЙ точки i
def build_2d_array_int_vectors(maxc, list_int_vectors_arrays):
    row = len(list_int_vectors_arrays)
    col = maxc

    a = np.array([[-1] * col for i in range(row)], dtype=int)
    for i in range(row):
        temp = np.array([-1 for i in range(col)], dtype=int)
        temp[:len(list_int_vectors_arrays[i])] = list_int_vectors_arrays[i]
        a[i] = temp
    
    return a

array_2d_new_old_point = build_2d_array_int_vectors(maxc, list_int_vectors_arrays)

In [9]:
array_2d_new_old_point[8:15, :]

array([[10044, 10045,    -1,    -1,    -1,    -1],
       [10042, 10043,    -1,    -1,    -1,    -1],
       [10040, 10041,    -1,    -1,    -1,    -1],
       [10038, 10039,    -1,    -1,    -1,    -1],
       [10034, 10035,    -1,    -1,    -1,    -1],
       [10031,    -1,    -1,    -1,    -1,    -1],
       [10028,    -1,    -1,    -1,    -1,    -1]])

In [10]:
# пересчитываем points2instances в соответствии с новой нумерацией точек
def recalc_points2instances(points2instances, array_2d_new_old_point):
    upd_points2instances = np.zeros((len(array_2d_new_old_point), 3), dtype=int)
    
    for i in range(len(upd_points2instances)):
        # находим значения в i--той строке --- это СТАРЫЕ точки, которые были объединены
        solutions = np.argwhere(array_2d_new_old_point[i] != -1)

        # пока что считаем, что из всех объединенных старых точек в одной строке для получения инстансов можно взять любую
        old_point_index = array_2d_new_old_point[i][solutions[0][0]]
        if old_point_index == 346: # для теста
            print(i)
        upd_points2instances[i] = points2instances[old_point_index]

    return upd_points2instances

upd_points2instances = recalc_points2instances(points2instances, array_2d_new_old_point)


2054


In [11]:
# проверяем корректность перенумерации
print(points2instances[346])
print(upd_points2instances[2054])
print(downpcd_trace[2][2054])

[12 11 13]
[12 11 13]
IntVector[346]


In [12]:
import numpy as np

from scipy.spatial.distance import cdist
from utils.distances_utils import sam_label_distance

downpcd = downpcd_trace[0] # облако после voxel_down_sample
points = np.asarray(downpcd.points)
spatial_distance = cdist(points, points)

print("len(downpcd.points) = {}".format(len(downpcd.points)))
print("len(upd_points2instances) = {}".format(len(upd_points2instances)))
dist, masks = sam_label_distance(upd_points2instances, spatial_distance, 2, 10)

len(downpcd.points) = 7307
len(upd_points2instances) = 7307


In [13]:
from services.normalized_cut_service import normalized_cut

T = 0.3
clusters = normalized_cut(dist, np.asarray(points), T)

/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 3 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)
/Users/sofiavivdich/proj/lidar-labelling/services/normalized_cut_service.py:27: RuntimeWarning: invalid value encountered in scalar divide
  return (cost / assoc_a) + (cost / assoc_b)
/Library/Python/3.9/site-packages/scipy/sparse/linalg/_eigen/arpack/arpack.py:939: LinAlgWarning: Diagonal number 4 is exactly zero. Singular matrix.
  self.M_lu = lu_factor(M)


In [14]:
len(clusters)

15

In [20]:
# import open3d as o3d

# from utils.pcd_utils import color_pcd_by_clusters

# pcd_clustered = color_pcd_by_clusters(downpcd, clusters)
# o3d.visualization.draw_geometries([pcd_clustered])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [15]:
import numpy as np
from scipy.spatial import KDTree

# cloud1 = np.asarray(downpcd.points)
# cloud2 = np.asarray(map.points)

cloud1 = np.asarray(map.points)
cloud2 = np.asarray(downpcd.points)

# Build KD-trees for each point cloud
tree1 = KDTree(cloud1)
tree2 = KDTree(cloud2)

# Find nearest neighbors for each point in cloud1
dists, indices = tree2.query(cloud1, k=1)

# Get the neighbor points from cloud2
neighbors = cloud2[indices]

In [16]:
map.points[4440]

array([23.21943973,  1.08491573, -1.46479947])

In [17]:
neighbors[4440]

array([23.21525976,  1.07515312, -1.46375785])

In [18]:
downpcd.points[4440]

array([22.71253833,  0.9524286 , -1.48947941])

In [19]:
# from utils.pcd_utils import color_pcd_by_clusters_and_neighbors

# import open3d as o3d

# pcd_clustered = color_pcd_by_clusters_and_neighbors(map, clusters, neighbors)
# o3d.visualization.draw_geometries([pcd_clustered])

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [19]:
from utils.pcd_utils import color_pcd_by_clusters_common

import open3d as o3d

pcd_clustered1, pcd_clustered2, counter1, counter2 = color_pcd_by_clusters_common(map, clusters, neighbors)
print("counter1 = {}, counter2 = {}".format(counter1, counter2))
print("removed points = {}".format(len(map.points) - len(downpcd.points)))
print("len(map.points) = {}".format(len(map.points)))
print("len(downpcd.points) = {}".format(len(downpcd.points)))
o3d.visualization.draw_geometries([pcd_clustered1])
o3d.visualization.draw_geometries([pcd_clustered2])

counter1 = 5124, counter2 = 5124
removed points = 2819
len(map.points) = 10126
len(downpcd.points) = 7307
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [21]:
c = 0
for i in range(len(clusters)):
    cluster = clusters[i]
    c += len(cluster)


In [22]:
c

7307

In [25]:
solutions = np.argwhere(array_2d_new_old_point[8] != -1)
len(solutions)

2

In [30]:
n = 0
m = 0
for i in range(len(upd_points2instances)):
    solutions = np.argwhere(array_2d_new_old_point[i] != -1)
    if len(solutions) > 1:
        n += len(solutions) - 1
        m += len(solutions)

In [31]:
n

2819

In [32]:
m

5124